In [18]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
import os

genai.configure(
    api_key=os.environ['API_KEY'])
model_gemini = genai.GenerativeModel(
    model_name='gemini-pro')


In [19]:
# import docx2txt
import time
# import pandas
import docx
import re


# Function to apply gemini-3.5 model to a given text

In [20]:
def generate_text(work, rubrics):
    rubrics = rubrics
    text = work
    instructor = """an experienced language instructor and assess the work of students. 
    I will provide the student's essay and the rubrics. 
    Your task is to assign a score and provide detailed comments for each rubric. 
    Highlight words, sentences, or phrases where the student made mistakes or did not meet the rubric criteria."""

    prompt_aspect = f"""Act as {instructor} 
    Rubrics:
    {rubrics}

    Student Work: 
    {text}

    Instructions:
    1. Assign a score for each rubric criterion based on the student's essay.
    2. Provide detailed comments explaining the score for each criterion.
    3. Highlight specific mistakes or areas where the essay did not meet the rubric criteria.
    4. Offer constructive feedback for improvement where applicable."""

    try:
        completion = model_gemini.generate_content(
            prompt_aspect,
            generation_config={
                'temperature': 0.1,
                'max_output_tokens': 800
            }
        )
        score_gemini = completion.text
    except Exception as e:
        score_gemini = str(e)
    
    result = score_gemini
    return result


# Functions to read files

In [21]:
# Define function to read text from a given Word file
def read_word_file(filename):

    # Open the Word file
    doc = docx.Document(filename)

    # Extract the text from the Word file
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)

    # Print the text from the Word file
    return "\n".join(text)

# Define function to read text from a given PDF file
def read_pdf_file(filename):
    with open(filename, 'rb') as f:
        reader = PyPDF2.PdfFileReader(f)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text



# Set essay id and Folder location

# Read Rubrics and Descriptions

In [22]:
import PyPDF2

# Replace 'data_path' with the actual path to your folder containing the PDF
pdf_file = "Rubric_ Holistic Essay Scoring.pdf"

try:
  # Open the PDF file
  with open(pdf_file, 'rb') as pdf_obj:
      pdf_reader = PyPDF2.PdfReader(pdf_obj)

      # Get number of pages
      num_pages = len(pdf_reader.pages)
      print(f"The PDF has {num_pages} pages.")

      # Extract text (assuming it's a text PDF)
      text = ""
      for page_num in range(2,num_pages):
          page = pdf_reader.pages[page_num]
          text += page.extract_text()
      rubrics = text
      print(rubrics)

except Exception as e:
  print(f"An error occurred: {e}")


The PDF has 4 pages.
Holistic
Rating
for
Independent
Writing
After
reading
each
essay
and
completing
the
analytical
rating
form,
assign
a
holistic
score
based
on
the
rubric
below .
For
the
following
evaluations
you
will
need
to
use
a
grading
scale
between
1
(minimum)
and
6
(maximum).
As
with
the
analytical
rating
form,
the
distance
between
each
grade
(e.g.,
1-2,
3-4,
4-5)
should
be
considered
equal.
SCORE
OF
6:
An
essay
in
this
category
demonstrates
clear
and
consistent
mastery
,
although
it
may
have
a
few
minor
errors.
A
typical
essay
effectively
and
insightfully
develops
a
point
of
view
on
the
issue
and
demonstrates
outstanding
critical
thinking,
using
clearly
appropriate
examples,
reasons,
and
other
evidence
to
support
its
position;
the
essay
is
well
organized
and
clearly
focused,
demonstrating
clear
coherence
and
smooth
progression
of
ideas;
the
essay
exhibits
skillful
use
of
language,
using
a
varied,
accurate,
and
apt
vocabulary
and
demonstrates
meaningful
variety
in
sentence
stru

# Call the process_files function

In [23]:

# Create a list of unique rubric names
unique_rubrics = ["Score"]
# Create a new dictionary for each unique rubric name to store the assessment results
rubric_results = {rubric: [] for rubric in unique_rubrics}
# df.head(5)
keywords = ["overall statement", "Comments", "Mistakes"]
results = {rubric: [] for rubric in keywords}
feedback= []


## Loop to Iterate through the CSV

In [24]:
import pandas as pd
if(os.path.isfile("results_gemini/Holistic_assessment_results_by_rubric.csv")):
    temp = pd.read_csv("results_gemini/Holistic_assessment_results_by_rubric.csv")
else:
    temp=[]
# Load the data from a CSV file
df = pd.read_csv("train.csv")
print(len(df))
print(len(temp))
print(len(df) - len(temp))
df = df[~df['essay_id'].isin(temp['essay_id'])]
df = df[df['score'] == 5]
print("after slicing", len(df))
df = df.dropna(axis=1, how='all')
df.head(2)

17307
4063
13244
after slicing 701


,essay_id,full_text,score
4704,46369ce,"""The Challenge of Exploring Venus"" argues why ...",5
4733,46b1195,"You get upearly on November 6, 2012 ready to v...",5


In [16]:
jump=2
csv_file_path = "results_gemini/Holistic_assessment_results_by_rubric.csv";
start = len(pd.read_csv(csv_file_path)) if os.path.exists(csv_file_path) else 0
print(start)

4959


In [17]:

for i in range(0, len(df), jump):
    data = df[i:i+jump] 
    # Iterate over the rows in the data
    for index, row in data.iterrows():

        # Get the text to assess
        text_to_assess = row["full_text"]    
        print(text_to_assess)
        # Apply the OpenAI model to the text to get the assessment result
        assessment_result = generate_text(work= text_to_assess, rubrics=rubrics)
        feedback.append(assessment_result)    
        print(assessment_result)
        # Extract the rubric values from the assessment result text
        for rubric in unique_rubrics:
            pattern = re.compile(rf"(?i){rubric}\D+(\d+(\.\d+)?)")
            match = pattern.search(assessment_result)
            print(match)
            if match:
                rubric_results[rubric].append(match.group(1))
            else:
                rubric_results[rubric].append("")
        for i in range(len(keywords)):
            keyword = keywords[i]
            pattern = re.compile(rf"(?i)\b{keyword}\b(.+?)(?=\b{keywords[i+1]}\b|$)" if i < len(keywords) - 1 else rf"(?i)\b{keyword}\b(.+?$)", re.DOTALL)
            match = pattern.search(assessment_result)
            if match:
                results[keyword].append(match.group(1).strip())
            else:
                results[keyword].append("")
        time.sleep(10)            
        print(rubric_results)

    for rubric in unique_rubrics:
        data[rubric+"_gemini"] = rubric_results[rubric]
    for keyword in keywords:
        data[keyword+"_gemini"] = results[keyword]
    data['Response_gemini'] = feedback
    # Save the data with the rubric columns to a new CSV file
    data.to_csv("results_gemini/Holistic_assessment_results_by_rubric.csv", index = False, mode='a', header=False)
    # data.to_csv("results_gemini/Holistic_assessment_results_by_rubric.csv", index = False)
    rubric_results = {rubric: [] for rubric in unique_rubrics}
    results = {rubric: [] for rubric in keywords}
    feedback=[]

Voting. Voting is a pretty big deal that only comes around so many years. And with voting the citizens of the U.S. have a voice. And also the voting method in which the president is elected by popular votes is just fine and it should be the only way people vote for their president. No one else should vote for them. The people have the right to choose who ever they want. And no one has the right to vote for any other person but them. The popular is just fine because first of all the popular vote for president method is more fair than the electoral college voting method. Secondly, the popular vote for president is less complicated, where as the electoral college can cause complications and problems.

To start off, the popular vote for president is more fair than the electoral college. First, there really is no way of cheating in the popular vote method because people vote once for whom ever they want to be president out of the people elected and then the president is chosen that way. The

C:\Users\Nimra\AppData\Local\Temp\ipykernel_11812\2742529503.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[rubric+"_gemini"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11812\2742529503.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[keyword+"_gemini"] = results[keyword]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11812\2742529503.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

**Holistic Score: 4**

**Content and Development**

* **Score: 4**
* The essay effectively develops a point of view on the issue and demonstrates competent critical thinking.
* The student provides adequate examples, reasons, and evidence to support their position.
* The essay is generally organized and focused, demonstrating some coherence and progression of ideas.

**Organization**

* **Score: 4**
* The essay is generally organized and focused, with a clear introduction, body paragraphs, and conclusion.
* The student uses transitions to connect ideas and paragraphs.
* However, there are some minor lapses in coherence and progression of ideas, such as in the second body paragraph where the student jumps from discussing the commonality of Martian mesas to the 2001 NASA photo.

**Language Use**

* **Score: 4**
* The essay exhibits adequate facility in the use of language.
* The student uses generally appropriate vocabulary and demonstrates some variety in sentence structure.
* However, 

C:\Users\Nimra\AppData\Local\Temp\ipykernel_11812\2742529503.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[rubric+"_gemini"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11812\2742529503.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[keyword+"_gemini"] = results[keyword]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11812\2742529503.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

**Holistic Score: 3**

**Analytical Rating Form**

**1. Development of a Point of View**

* **Score: 3**
* **Comments:** The essay develops a clear point of view that the electoral vote system is flawed and should be replaced. However, the essay does not consistently support this point of view with strong evidence and analysis.

**2. Critical Thinking**

* **Score: 3**
* **Comments:** The essay demonstrates some critical thinking skills, such as identifying potential problems with the electoral vote system. However, the essay does not consistently provide in-depth analysis or consider alternative perspectives.

**3. Use of Evidence**

* **Score: 2**
* **Comments:** The essay provides some examples to support its claims, but the evidence is not always relevant or sufficient. The essay also relies heavily on generalizations and does not provide specific data or research to support its arguments.

**4. Organization and Focus**

* **Score: 3**
* **Comments:** The essay is generally organiz

...

C:\Users\Nimra\AppData\Local\Temp\ipykernel_11812\2742529503.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[rubric+"_gemini"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11812\2742529503.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[keyword+"_gemini"] = results[keyword]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11812\2742529503.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

**Holistic Score: 4**

**Content and Development**

* **Score: 4**
* **Comments:** The essay effectively develops a point of view on the issue and demonstrates competent critical thinking. The student provides adequate examples, reasons, and evidence to support their position. However, the essay could benefit from more in-depth analysis and exploration of the topic.

**Organization and Focus**

* **Score: 4**
* **Comments:** The essay is generally well-organized and focused. The student presents their arguments in a logical order and provides clear transitions between paragraphs. However, the essay could benefit from a stronger introduction and conclusion that more effectively summarizes the main points.

**Language Use**

* **Score: 4**
* **Comments:** The essay exhibits adequate facility in the use of language. The student uses appropriate vocabulary and demonstrates some variety in sentence structure. However, the essay could benefit from more precise word choice and a more varied u

C:\Users\Nimra\AppData\Local\Temp\ipykernel_11812\2742529503.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[rubric+"_gemini"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11812\2742529503.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[keyword+"_gemini"] = results[keyword]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11812\2742529503.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

**Holistic Score: 5**

**Content and Development**

* **Score: 5**
* The essay effectively develops a point of view on the issue and demonstrates strong critical thinking.
* The author provides compelling evidence to support their argument, including the benefits of exploring Venus, ways to avoid dangers, and future experiments.
* The essay is well-organized and focused, with a clear progression of ideas.

**Language Use**

* **Score: 5**
* The essay exhibits facility in the use of language, using appropriate vocabulary and demonstrating variety in sentence structure.
* The author uses strong hooks and compelling arguments to captivate the reader.

**Grammar, Usage, and Mechanics**

* **Score: 5**
* The essay is generally free of errors in grammar, usage, and mechanics.

**Specific Feedback**

**Content and Development**

* The author could strengthen their argument by providing more specific examples of the benefits of exploring Venus.
* The author could also discuss the potential ris

KeyboardInterrupt: 